In [10]:
import numpy as np
import copy

In [11]:
EPSILON = 0.1
N_EXPERIENCES = 50000
MIN_ALPHA = 0.02
ALPHAS = np.linspace(1.0, MIN_ALPHA, N_EXPERIENCES)
GAMMA = 0.8
EPSILON = 0.4
ACTION_SIZE = 9
MOVES = ['_', 'X', 'O']
PLAYER1 = 1
PLAYER2 = 2
PLAYER1_MOVE = 1
PLAYER2_MOVE = 4

WIN_REWARD = 50
SMART_REWARD = 20
MOVE_REWARD = -5
LOST_REWARD = -1000
DRAW_REWARD = 30

In [12]:
# State: a string represents a matrix 3x3 contains _ (empty), X, O
def de_state(state_str):
  # TODO: return a 2D array for a string
  state = np.zeros(ACTION_SIZE)
  i = 0
  for move in state_str:
    state[i] = 0 if move == '_' else PLAYER1_MOVE if move == 'X' else PLAYER2_MOVE
    i += 1
  state = np.reshape(state, (3, 3))
  return state

In [13]:
class GameAgent:
  def __init__(self):
    self.q_table = {}
  
  def random_policy(self, state):
    # TODO: from a state, return a random move
    valid_move = False
    while not valid_move:
      move = np.random.randint(ACTION_SIZE)
      valid_move = state[move] == '_'
    return move
  
  # return the action that have Q(state, action) max
  # if action is not valid, return random action
  def policy(self, state):
    # action = np.argmax(self.Q(state))
    
    # if state[action] != '_':
    #   action = self.random_policy(state)
    sorted_actions = np.argsort(self.Q(state))
    for i in range(ACTION_SIZE):
      if state[sorted_actions[ACTION_SIZE - i - 1]] == '_':
        return sorted_actions[ACTION_SIZE - i - 1]
    return self.random_policy(state)
  # Choose an action, either random or by policy learnt after trained
  # DO NOT change this method
  def choose_action(self, state):
    if np.random.random() < EPSILON:
        return self.random_policy(state)
    return self.policy(state)
  
  # return the Q value based on state & action: Q(state, action)
  # DO NOT change this method
  def Q(self, state, action=None):
    # if it's a new state, add to the q_table dictionary with all Q values to zeros
    if state not in self.q_table:
      self.q_table[state] = np.zeros(ACTION_SIZE)
    # if no action, return all q values
    if action is None:
      return self.q_table[state]    
    
    return self.q_table[state][action] # Return Qvalue of (state, action) pair

In [14]:
class Environment:
  def __init__(self):
    self.state = ''
    self.n_moves = 0
  
  def reset(self):
    self.state = '_________'
    self.n_moves = 0
    return self.state
  
  def is_draw(self):
    return '_' not in self.state and not self.is_win()

  def is_win(self):
    # TODO:
    # check if current state is a game over (having 3 consecutive Xs or Os)
    state_matrix = de_state(self.state)

    # Check rows
    if sum(state_matrix[0]) == 3 or sum(state_matrix[1]) == 3 or sum(state_matrix[2]) == 3:
      return True
    if sum(state_matrix[0]) == 12 or sum(state_matrix[1]) == 12 or sum(state_matrix[2]) == 12:
      return True
    
    # Check columns
    if sum(state_matrix[:, 0]) == 3 or sum(state_matrix[:, 1]) == 3 or sum(state_matrix[:, 2]) == 3:
      return True
    if sum(state_matrix[:, 0]) == 12 or sum(state_matrix[:, 1]) == 12 or sum(state_matrix[:, 2]) == 12:
      return True

    # Check diagonals
    d1 = [state_matrix[i][i] for i in range(3)]
    d2 = [state_matrix[i][2 - i] for i in range(3)]
    if sum(d1) == 3 or sum(d1) == 12 or sum(d2) == 3 or sum(d2) == 12:
      return True
    
    return False
  
  def update_state(self, action, player):
    # update state with action
    state_list = [move for move in self.state]
    state_list[action] = MOVES[player]
    self.state = ''.join(state_list)

  def perform(self, action, player):
    self.n_moves += 1
    n_2consecutives = self.count_consecutives()
    self.update_state(action, player)
    n_new_2consecutives = self.count_consecutives()
    
    win = self.is_win()
    draw = self.is_draw()

    done = win or draw

    reward = WIN_REWARD if win else DRAW_REWARD if draw else SMART_REWARD * (n_new_2consecutives - n_2consecutives)
    reward += MOVE_REWARD

    return self.state, reward, done
  
  def show_state(self):
    i = 0
    for move in self.state:
      print(move, end='')
      i += 1
      if i % 3 == 0:
        print()

  def count_consecutives(self):
    state_matrix = de_state(self.state)
    count = 0
    for row in range(len(state_matrix)):
      count += 1 if state_matrix[row, 0] == state_matrix[row, 1] == PLAYER1_MOVE else \
               1 if state_matrix[row, 0] == state_matrix[row, 1] == PLAYER2_MOVE else 0
      count += 1 if state_matrix[row, 1] == state_matrix[row, 2] == PLAYER1_MOVE else \
               1 if state_matrix[row, 1] == state_matrix[row, 2] == PLAYER2_MOVE else 0
    for col in range(len(state_matrix)):
      count += 1 if state_matrix[0, col] == state_matrix[1, col] == PLAYER1_MOVE else \
               1 if state_matrix[0, col] == state_matrix[1, col] == PLAYER2_MOVE else 0
      count += 1 if state_matrix[1, col] == state_matrix[2, col] == PLAYER1_MOVE else \
               1 if state_matrix[1, col] == state_matrix[2, col] == PLAYER2_MOVE else 0
    
    count += 1 if state_matrix[0, 0] == state_matrix[1, 1] == PLAYER1_MOVE else \
             1 if state_matrix[0, 0] == state_matrix[1, 1] == PLAYER2_MOVE else 0
    count += 1 if state_matrix[1, 1] == state_matrix[2, 2] == PLAYER1_MOVE else \
             1 if state_matrix[1, 1] == state_matrix[2, 2] == PLAYER2_MOVE else 0
    count += 1 if state_matrix[0, 2] == state_matrix[1, 1] == PLAYER1_MOVE else \
             1 if state_matrix[0, 2] == state_matrix[1, 1] == PLAYER2_MOVE else 0
    count += 1 if state_matrix[1, 1] == state_matrix[2, 0] == PLAYER1_MOVE else \
             1 if state_matrix[1, 1] == state_matrix[2, 0] == PLAYER2_MOVE else 0
    
    return count

In [15]:
class TicTacToeModel:
  def train(self, verbose=False):
    player1 = GameAgent()
    player2 = GameAgent()
    env = Environment()

    for exp in range(N_EXPERIENCES//2):
      self.training(env, player1, player2, exp, verbose)
    for exp in range(N_EXPERIENCES//2, N_EXPERIENCES):
      self.training(env, player2, player1, exp, verbose)
    
    with open('model.txt', 'w') as f:
      for state in player1.q_table.keys():
          f.write(state + '\n')
          for val in player1.q_table[state]:
              f.write(str(val) + ',')
          f.write('\n')

    return player1 # doesn't matter player 1 or 2 if number of experiences are large enough
  def training(self, env, player1, player2, exp, verbose=False):
    #for exp in range(N_EXPERIENCES):
      state = env.reset()
      player1_rewards = 0
      player2_rewards = 0
      alpha = ALPHAS[exp]
      done = False

      while not done:
        old_state1 = env.state
        next_state1, reward, action1, done = self.train_one_step(env, player1, PLAYER1, alpha)
        player1_rewards += reward
        if done:
          # Train player 2 with LOST_REWARD
          self.train_when_loose(env, player2, next_state2, old_state2, action2, alpha) 
        else:
          old_state2 = env.state
          next_state2, reward, action2, done = self.train_one_step(env, player2, PLAYER2, alpha)
          player2_rewards += reward
          if done:
            self.train_when_loose(env, player1, next_state1, old_state1, action1, alpha) # Train player 1 because it's lost
        
      if verbose:
        print('Experience {0} - P1 reward: {1} - P2 reward: {2}'.format(exp, player1_rewards, player2_rewards))
      return player1, player2

  def train_when_loose(self, env, player, next_state, state, action, alpha):
    reward = LOST_REWARD
    player.Q(state)[action] = player.Q(state, action) + alpha * \
                                        (reward + GAMMA *  np.max(player.Q(next_state)) - player.Q(state, action))

  def train_one_step(self, env, player, player_order, alpha):
    state = env.state
    action = player.choose_action(state)
    next_state, reward, done = env.perform(action, player_order)
    
    player.Q(state)[action] = player.Q(state, action) + alpha * \
                                        (reward + GAMMA *  np.max(player.Q(next_state)) - player.Q(state, action))

    return next_state, reward, action, done

  def play(self, player1, player2):
    env = Environment()
    state = env.reset()
    done = False

    env.show_state()

    while not done:
      print('{0} moves'.format(player1.name))
      action = player1.move(env.state)
      env.update_state(action, PLAYER1)
      env.show_state()
      
      if env.is_win():
        print('{0} wins!'.format(player1.name))
      elif not env.is_draw():
        print('{0} moves'.format(player2.name))
        action = player2.move(env.state)
        env.update_state(action, PLAYER2)
        env.show_state()

        if env.is_win():
          print('{0} wins!'.format(player2.name))
      
      if env.is_draw():
        print('Draw game!')

      done = env.is_win() or env.is_draw()

In [16]:
class Computer:
  def __init__(self, agent, name):
    self.agent = agent
    self.name = name
  def move(self, state):
    return agent.policy(state)

class Human:
  def __init__(self, name):
    self.name = name

  def move(self, state):
    x = y = -1
    invalid_move = True
    while invalid_move:
      xy = input('')
      x = int(xy.split(',')[0])
      y = int(xy.split(',')[1])
      move = x * 3 + y
      invalid_move = x < 0 or x > 2 or y < 0 or y > 2 or state[move] != '_'
      if invalid_move:
        print('Invalid move, try again!')
    return move

In [17]:
model = TicTacToeModel()
agent = model.train(verbose=True)

Experience 0 - P1 reward: 35 - P2 reward: 10
Experience 1 - P1 reward: 20 - P2 reward: 70
Experience 2 - P1 reward: 25 - P2 reward: 40
Experience 3 - P1 reward: 50 - P2 reward: 5
Experience 4 - P1 reward: 50 - P2 reward: 45
Experience 5 - P1 reward: 70 - P2 reward: -15
Experience 6 - P1 reward: 60 - P2 reward: 50
Experience 7 - P1 reward: 50 - P2 reward: 5
Experience 8 - P1 reward: 25 - P2 reward: 40
Experience 9 - P1 reward: 30 - P2 reward: -15
Experience 10 - P1 reward: 55 - P2 reward: 10
Experience 11 - P1 reward: 125 - P2 reward: 20
Experience 12 - P1 reward: 105 - P2 reward: 20
Experience 13 - P1 reward: 85 - P2 reward: 20
Experience 14 - P1 reward: 60 - P2 reward: 30
Experience 15 - P1 reward: 30 - P2 reward: -15
Experience 16 - P1 reward: 90 - P2 reward: 25
Experience 17 - P1 reward: 45 - P2 reward: 60
Experience 18 - P1 reward: 65 - P2 reward: 80
Experience 19 - P1 reward: 45 - P2 reward: 60
Experience 20 - P1 reward: 50 - P2 reward: 25
Experience 21 - P1 reward: 70 - P2 reward

In [18]:
player1 = Computer(agent, 'Agent')
player2 = Human('Human')
model.play(player1, player2)

___
___
___
Agent moves
___
___
__X
Human moves


ValueError: invalid literal for int() with base 10: ''